In [11]:
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [12]:
EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4


In [ ]:
def load_data(data_dir):
    images = []
    labels = []

    for category in range(NUM_CATEGORIES):
        path = os.path.join(data_dir, str(category))
        if not os.path.isdir(path):
            continue
        try:
            for img_file in os.listdir(path):
                if img_file.startswith('.'):
                    continue
                img_path = os.path.join(path, img_file)
                img_array = cv2.imread(img_path)
                if img_array is None:
                    continue
                img_array = cv2.resize(img_array, (IMG_WIDTH, IMG_HEIGHT))
                images.append(img_array)
                labels.append(category)
        except OSError:
            continue
    if not images:
        raise ValueError(f"No valid images found in {data_dir}")

    return (images, labels)

In [14]:
def get_model():
    model = tf.keras.Sequential([

        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),


        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),


        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [16]:
images, labels = load_data('./Data/gtsrb')
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train = np.array(x_train, dtype='float32') / 255.0
x_test = np.array(x_test, dtype='float32') / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)
model = get_model()
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy}")


Epoch 1/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.1789 - loss: 3.0858 - val_accuracy: 0.8121 - val_loss: 0.7232
Epoch 2/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.7572 - loss: 0.7715 - val_accuracy: 0.9396 - val_loss: 0.2330
Epoch 3/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.8936 - loss: 0.3513 - val_accuracy: 0.9628 - val_loss: 0.1315
Epoch 4/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9364 - loss: 0.2054 - val_accuracy: 0.9767 - val_loss: 0.0924
Epoch 5/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9548 - loss: 0.1431 - val_accuracy: 0.9844 - val_loss: 0.0623
Epoch 6/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9665 - loss: 0.1109 - val_accuracy: 0.9792 - val_loss: 0.0797
Epoch 7/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9713 - loss: 0.0949 - val_accuracy: 0.9842 - val_loss: 0.0591
Epoch 8/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9771 - loss: 0.0716 - val_accu

In [17]:
model.save("traffic_cnn.h5")